### Import packages we need

In [1]:
import numpy as np
import pandas as pd
import math
pd.options.mode.chained_assignment = None 
import os
cwd = os.getcwd()
import nltk
from nltk.tokenize import word_tokenize
import collections
import re
import operator
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import math

### Read in the data and calcuate average price per person per night of a rental based on number of accommodation 

In [64]:
Price = pd.read_csv('listings.csv')

In [321]:
Price['price'] = Price['price'].str.replace('$','')
Price['price'] = Price['price'].str.replace(',','')
Price['price'] = pd.to_numeric(Price['price'])
Price['accommodates'] = pd.to_numeric(Price['accommodates'])
price = Price.dropna(subset=['transit'])
Price_94104 = Price[Price['zipcode'] == float(94104)].dropna(subset=['description'])

#calculate the average price based on the price and how many people a room can accommodate
price['average price'] = price['price']/price['accommodates']
Price['average price'] = Price['price']/Price['accommodates']

### create filler word dictionary

In [162]:
stop_words = set(stopwords.words('english')) 

### count the frequency of each word being used in transit descripitions 

In [221]:
def count_word(price,stop_words,column,n):
    dict_full_list = []
    string = str(list(price[column]))
    string_replace = re.sub('[0-9]|\,|\.|\)|\(|\'|-|:|\``|\#|\&|’|\/|\$|\?|\!|;|•|-|\*|~|>|=|–|(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)','',string).lower()
    word_tokens = map(str.lower,word_tokenize(string_replace))
    word_tokens = [w for w in word_tokens if w not in stop_words]
    di = dict(collections.Counter(word_tokens))
    sorted_di = dict(sorted(di.items(), key=operator.itemgetter(1),reverse=True))
    for j in range(len(sorted_di.keys())):
        sorted_di[list(sorted_di.keys())[j]] = sorted_di[list(sorted_di.keys())[j]]
    try:
        del sorted_di["``"]
        del sorted_di["''"]
    except:
        pass
    
    dict_full_list.append(sorted_di)
    
    dict_full_list = dict_full_list[0]
    keys = list(dict_full_list.keys())
    
    words = []
    freq = []
    for i in range(0,n):
        if keys[i] !="``" and "''":
            words.append(keys[i])
            freq.append(dict_full_list[keys[i]])
                 
    word_counts = pd.DataFrame({"words":words,"freq":freq})
    
    return word_counts


In [241]:
column = 'transit'
word_counts_transit = count_word(price,stop_words,column,400)

column = 'description'
word_counts_description = count_word(price,stop_words,column,400)
word_counts_94014 = count_word(Price_94104,stop_words,column,400)


column = 'amenities'
word_counts_amenities = count_word(price,stop_words,column,400)

In [242]:
word_counts_transit.to_csv("word_counts_transit.csv")
word_counts_description.to_csv("word_counts_description.csv")
word_counts_94014.to_csv("word_counts_94104.csv")

### breakdown the average price interval into every 10 dollars and count the word frequency of transit descrption in each price range

In [166]:
def create_dict(price):
    dict_list = {}
    for i in range(4):
        priceCap = float(i * 50)
        p = price.copy()
        p =p[(p['price'] > priceCap) & (p['price'] <= priceCap + 50)]

        string = str(list(p["transit"]))
        string_replace = re.sub('[0-9]|\,|\.|\)|\(|\]|\[|\'|-|:|\``|\#|\&|’|\/|\$|\?|\!|;|•|-|\*|~|>|=|–|(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)','',string).lower()
        word_tokens = map(str.lower,word_tokenize(string_replace))
        word_tokens = [w for w in word_tokens if w not in stop_words]
        di = dict(collections.Counter(word_tokens))
        sorted_di = dict(sorted(di.items(), key=operator.itemgetter(1),reverse=True))
        for j in range(len(sorted_di.keys())):
            sorted_di[list(sorted_di.keys())[j]] = sorted_di[list(sorted_di.keys())[j]]/len(p)
        del sorted_di["``"]
        del sorted_di["''"]
        dict_list[i] = sorted_di


    p = price.copy()
    p = p[(p['average price'] > 200)]
    string = str(list(p["transit"]))
    string_replace = re.sub('[0-9]|\,|\.|\)|\(|\'|-|:|\]|\[|\``|\#|\&|’|\/|\$|\?|\!|;|•|-|\*|~|>|=|–|(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)','',string).lower()
    word_tokens = map(str.lower,word_tokenize(string_replace))
    word_tokens = [w for w in word_tokens if w not in stop_words]
    di = dict(collections.Counter(word_tokens))
    sorted_di = dict(sorted(di.items(), key=operator.itemgetter(1),reverse=True))
    for j in range(len(sorted_di.keys())):
        sorted_di[list(sorted_di.keys())[j]] = sorted_di[list(sorted_di.keys())[j]]/len(p)

    del sorted_di["``"]
    del sorted_di["''"]
    dict_list[4] = sorted_di
    
    return dict_list


In [167]:
dict_list = create_dict(price)

### list of means of transportation 

In [311]:
checkBus = ["bus","bart","shuttle","car","bike","train","walk","uber","lyft","taxi"]
checkBus_ = ["parking"]
checkPet = ["pets","Pets","Dogs","Dog","Cat","Cats"]
checkDescription = ["private","kitchen"]

In [312]:
def create_Price_Range_Word_Freq(dict_list):
    word = []
    price_Range = []
    freq = []
    for i in range(len(dict_list.keys())):
        keys = list(dict_list[i].keys())
        for j in range(11):
            word.append(keys[j])
            price_Range.append(str(i * 50) + "~" + str((i + 1) * 50))
            freq.append(dict_list[i][keys[j]])

    for i in range(1,len(price_Range)):
        if price_Range[i] == "200~250":
            price_Range[i] = ">200"
            
    word_freq_price_interval = pd.DataFrame({"word": word,"group":price_Range,"value":freq})
    
    return word_freq_price_interval
    

### wirte word frequency count csv

In [313]:
word_freq_price_interval = create_Price_Range_Word_Freq(dict_list)
word_freq_price_interval.to_csv("word_freq_price_interval.csv")

In [314]:
def get_Data_Frame(check,p,column):
    price = p.copy()
    for i in range(len(check)):
        price[check[i]] = 0
    for i in list(price[column].index):
        for j in range(len(check)):
            try:
                list_ = re.sub('[0-9]|\{|\}|\"|\,|\.|\)|\(|\]|\[|\'|-|:|\``|\#|\&|’|\/|\$|\?|\!|;|•|-|\*|~|>|=|–|(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)','',\
                              price[column][i]).lower().split()
                if check[j] in list_:
                    price[check[j]][i] = "Include"
                else:
                    price[check[j]][i] = "Not Include"
            except:
                price[check[j]][i] = "Not Include"
    return price

In [315]:
def get_Data_Frame_Amenities(check,p,column):
    price = p.copy()
    for i in range(len(check)):
        price[check[i]] = 0
    for i in list(price[column].index):
        for j in range(len(check)):
            list_ = price[column][0].replace("\""," ").replace("\'"," ").replace("-"," ").\
            replace("{"," ").replace("}"," ").lower().split(',')
            list_ = [list_.strip() for item in list_]
            if check[j] in list_:
                price[check[j]][i] = 1
            else:
                price[check[j]][i] = 0
    return price

In [316]:
def get_Data_Frame_Flag(check,p,column):
    price = p.copy()
    price["Transit"] = 0
    for i in range(len(price[column])):
        try:
            list_ = re.sub('[0-9]|\,|\.|\)|\(|\]|\[|\'|-|:|\``|\#|\&|’|\/|\$|\?|\!|;|•|-|\*|~|>|=|–|(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)','',\
                          price[column][i]).lower().split()
            if any(item for item in list_ if item in check):
                price["Transit"][i] = "Include" 
            else:
                price["Transit"][i] = "Not Include"
        except:
            price["Transit"][i] = "Not Include"
    return price

In [317]:
def get_Data_Frame_Flag_Parking(check,p,column):
    price = p.copy()
    price["Parking"] = 0
    for i in range(len(price[column])):
        try:
            if any(item for item in check if item in price[column][i]):
                price["Parking"][i] = "Include"
            else:
                price["Parking"][i] = "Not Include"
        except:
            price["Parking"][i] = "Not Include"
    return price

In [318]:
column = "transit"
priceNew = get_Data_Frame(checkBus,price,column)
price_New_Parking = get_Data_Frame_Flag_Parking(checkBus_,Price,column)
price_New_Transit = get_Data_Frame_Flag(checkBus,Price,column)

column = 'description'
price_New_Description = get_Data_Frame(checkDescription,Price,column)
#column = "amenities"
#price_New_Pets = get_Data_Frame_Amenities(checkPet,Price,column)

In [319]:
priceNew.to_csv("priceNew.csv")
price_New_Description.to_csv("price_New_Description.csv")
price_New_Parking.to_csv("price_New_Parking.csv")
price_New_Transit.to_csv("price_New_Transit.csv")
price_New_Pets.to_csv("price_New_Pets.csv")

In [65]:
for index,row in Price.iterrows():
    shortest_distance = 100
    for j in range(len(sight_coordinate)):
        distance = ((Price['latitude'][index] - sight_coordinate[j][0])**2 + (Price['longitude'][index] - sight_coordinate[j][1])**2) ** 1/2
        if distance < shortest_distance: 
            shortest_distance = distance
    Price.at[index,'Shortest Distance'] = shortest_distance         

In [67]:

Price.to_csv("price_Sight.csv")